# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [37]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [38]:
# your code here
data = pd.read_csv(r"C:\Users\mfduh\OneDrive\Documents\Deutschland\Arbeiten\Data Analyst\Jupyter excercises\Week 8 - 24.10.2022\8.4 Natural Language Processing\Sentiment140.csv")
# data
# shape = 1600000 rows × 6 columns

In [39]:
sample = data.sample(frac=0.002, random_state=1)
sample = sample.reset_index(drop=True)
sample.shape

(3200, 6)

In [40]:
sample.columns = ['target','id','date','flag','user','text']
sample.tail()

,target,id,date,flag,user,text
3195,4,1678633429,Sat May 02 07:17:59 PDT 2009,NO_QUERY,duzzzstar,"Aaaaaaaaao, myyyyyyy pretty faaaaaaace! sheeee..."
3196,4,1969775978,Sat May 30 01:11:01 PDT 2009,NO_QUERY,sarahnewton,@kimolver thanks so much for the #followfriday...
3197,4,1753237211,Sun May 10 00:42:05 PDT 2009,NO_QUERY,pinkllamas820,Made coffee cake for her mom
3198,4,2057189605,Sat Jun 06 12:32:22 PDT 2009,NO_QUERY,aprilmaeee,Reinvent myself this summer
3199,0,2016659122,Wed Jun 03 07:09:02 PDT 2009,NO_QUERY,rpattzlawyer,.@RPatZHood sorry about last night i know you...


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [41]:
%run challenge-1.ipynb import clean_up, tokenize, stem_and_lemmatize, remove_stopwords

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\mfduh\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mfduh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mfduh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [42]:
%%time
sample['text_processed'] = sample['text'].apply(clean_up).apply(tokenize).apply(stem_and_lemmatize).apply(remove_stopwords)

CPU times: total: 5.83 s
Wall time: 13.5 s


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [43]:
len(sample.text_processed)

3200

In [44]:
all_words = []

for i in range(len(sample.text_processed)):
    for j in range(len(sample.text_processed.iloc[i])):
        all_words.append(sample.text_processed.iloc[i][j].lower())
        
len(all_words)

24886

In [49]:
# all_words

In [46]:
from nltk.probability import FreqDist
freq_dist = nltk.FreqDist(all_words)
top_5000 = freq_dist.most_common(5000)

In [50]:
len(top_5000)
# top_5000

5000

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [48]:
# your code here
# this step is not necessary
word_features, freq = zip(*top_5000)

In [51]:
def find_features(df):
    word = set(df)
    features = {}
    for w in word_features:
        features[w] = (w in word)   # creates boolean, true/false
    return features

features = []
for i, l in enumerate(sample['text_processed']):
    s = [find_features(l),sample['target'].iloc[i]]
    z = tuple(s)
    features.append(z)
# features # this is for all tweets/rows

In [53]:
# this is only for 1 tweet/row
function_one = find_features(sample.text_processed.iloc[0]) 
# function_one

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [54]:
# first option, divide them equally
length = len(features)
print(length)

middle_index = length // 2

first_half = features[:middle_index]
second_half = features[middle_index:]

print(len(first_half))
print(len(second_half))

3200
1600
1600


In [55]:
# second option, divide them with a percentage

trainset = features[:round(len(features)*0.75)]
testset = features[:round(len(features)*0.25)]

In [56]:
# 0 - positive
# 4 - negative

# your code here
# split
train, test = first_half, second_half

In [57]:
classifier = nltk.NaiveBayesClassifier.train(train)
classifier2 = nltk.NaiveBayesClassifier.train(trainset)

In [58]:
classifier.show_most_informative_features()
classifier2.show_most_informative_features()

Most Informative Features
                    hate = True                0 : 4      =     13.6 : 1.0
                   thank = True                4 : 0      =     12.2 : 1.0
                    sick = True                0 : 4      =     10.3 : 1.0
                     die = True                0 : 4      =      7.6 : 1.0
                    hurt = True                0 : 4      =      7.6 : 1.0
                    bore = True                0 : 4      =      6.6 : 1.0
                       b = True                0 : 4      =      6.3 : 1.0
                    damn = True                0 : 4      =      6.3 : 1.0
                    miss = True                0 : 4      =      6.2 : 1.0
                   funni = True                4 : 0      =      5.7 : 1.0
Most Informative Features
                    hate = True                0 : 4      =     18.3 : 1.0
                    sick = True                0 : 4      =     10.6 : 1.0
                    kill = True                0

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [59]:
# your code here 
# check
print('class 50/50: ', nltk.classify.accuracy(classifier, test))
print('class 75/25: ',nltk.classify.accuracy(classifier2, testset))

class 50/50:  0.67625
class 75/25:  0.94125


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here